In [9]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env
import polars as pl

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# CPI + CLI DATA COLLECTION


## OECD Composite Leading Indicator

In [22]:
cli = obb.economy.composite_leading_indicator(start_date='2024-01-01', provider='oecd', country=['united_states'])
cli.to_polars()

date,value,country
date,f64,str
2024-01-01,99.5686,"""United States"""
2024-02-01,99.72427,"""United States"""
2024-03-01,99.85373,"""United States"""
2024-04-01,99.92516,"""United States"""
2024-05-01,99.94516,"""United States"""
2024-06-01,99.94624,"""United States"""
2024-07-01,99.96982,"""United States"""
2024-08-01,100.0283,"""United States"""
2024-09-01,100.1253,"""United States"""


## Consumer Price Index

### OECD Consumer Price Index

In [23]:
oecd_cpi = obb.economy.cpi(start_date='2010-01-01', country='united_states', transform='yoy', provider='oecd',harmonized=False, expenditure='total')
oecd_cpi.results


[OECDCPIData(date=2010-01-01, country=united_states, value=0.02625709, expenditure=total),
 OECDCPIData(date=2010-02-01, country=united_states, value=0.02143332, expenditure=total),
 OECDCPIData(date=2010-03-01, country=united_states, value=0.02313959, expenditure=total),
 OECDCPIData(date=2010-04-01, country=united_states, value=0.02236447, expenditure=total),
 OECDCPIData(date=2010-05-01, country=united_states, value=0.020209860000000003, expenditure=total),
 OECDCPIData(date=2010-06-01, country=united_states, value=0.010533490000000001, expenditure=total),
 OECDCPIData(date=2010-07-01, country=united_states, value=0.01235193, expenditure=total),
 OECDCPIData(date=2010-08-01, country=united_states, value=0.01148105, expenditure=total),
 OECDCPIData(date=2010-09-01, country=united_states, value=0.01143683, expenditure=total),
 OECDCPIData(date=2010-10-01, country=united_states, value=0.01172188, expenditure=total),
 OECDCPIData(date=2010-11-01, country=united_states, value=0.011431610

### FRED Consumer Price Index

In [24]:
fred_cpi = obb.economy.cpi(start_date='2024-07-01', country='united_states', transform='yoy', provider='fred')
fred_cpi.results

[FREDConsumerPriceIndexData(date=2024-07-01, country=united_states, value=0.028947529999999996),
 FREDConsumerPriceIndexData(date=2024-08-01, country=united_states, value=0.0253073),
 FREDConsumerPriceIndexData(date=2024-09-01, country=united_states, value=0.02440633),
 FREDConsumerPriceIndexData(date=2024-10-01, country=united_states, value=0.02597905)]

# humblCOMPASS 

In [22]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    # symbols=["AAPL"],
    # interval="1d",
    start_date="2024-01-01",
    end_date="2024-12-01",
    membership="humblPERMANENT"
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
compass = toolbox.fundamental.humbl_compass(
    country="turkey",
    z_score="3mo",
    chart=True,
    recommendations=True
)



INFO: HumblCompassFetcher || START: fetch_data (sync)
INFO: HumblCompassFetcher || CLI start date: 2023-06-01 and CPI start date: 2023-06-01. Dates are adjusted to account for CLI data release lag and z-score calculation window.
INFO: HumblCompassFetcher || END: fetch_data (sync) - Total time: 26.1945s


In [24]:
compass.to_polars()

date_month_start,country,cpi,cpi_3m_delta,cli,cli_3m_delta,humbl_regime,cpi_zscore,cli_zscore
date,str,f64,f64,f64,f64,str,f64,f64
2023-09-01,"""Turkey""",61.53,23.32,99.87,-1.12,"""humblBLOAT""",0.75,-1.02
2023-10-01,"""Turkey""",61.36,13.53,99.44,-1.27,"""humblBLOAT""",0.52,-0.99
2023-11-01,"""Turkey""",61.98,3.04,99.08,-1.24,"""humblBLOAT""",1.11,-0.97
2023-12-01,"""Turkey""",64.77,3.24,98.83,-1.04,"""humblBLOAT""",1.14,-0.94
2024-01-01,"""Turkey""",64.86,3.5,98.68,-0.76,"""humblBLOAT""",0.6,-0.91
…,…,…,…,…,…,…,…,…
2024-06-01,"""Turkey""",71.6,3.1,98.62,0.1,"""humblBOUNCE""",-0.24,1.06
2024-07-01,"""Turkey""",61.78,-8.02,98.68,0.17,"""humblBOOM""",-1.11,1.0
2024-08-01,"""Turkey""",51.97,-23.47,98.77,0.21,"""humblBOOM""",-1.0,1.04


In [13]:
compass.extra


{'humbl_regime_recommendations': {'asset_classes': {'best': [<AssetRecommendation.GOLD: 'Gold'>,
    <AssetRecommendation.COMMODITIES: 'Commodities'>],
   'worst': [<AssetRecommendation.CREDIT: 'Credit'>],
   'rationale': 'USD devaluation and money printing favor real assets'},
  'equity_sectors': {'best': [<AssetRecommendation.UTILITIES: 'Utilities'>,
    <AssetRecommendation.TECHNOLOGY: 'Technology'>,
    <AssetRecommendation.ENERGY: 'Energy'>,
    <AssetRecommendation.INDUSTRIALS: 'Industrials'>],
   'worst': [<AssetRecommendation.FINANCIALS: 'Financials'>,
    <AssetRecommendation.REITS: 'REITs'>,
    <AssetRecommendation.MATERIALS: 'Materials'>,
    <AssetRecommendation.TELECOM: 'Telecom'>],
   'rationale': 'Sectors with pricing power and real asset exposure outperform'},
  'equity_factors': {'best': [<AssetRecommendation.SECULAR_GROWTH: 'Secular Growth'>,
    <AssetRecommendation.MOMENTUM: 'Momentum'>,
    <AssetRecommendation.MID_CAPS: 'Mid Caps'>,
    <AssetRecommendation.LOW_B

In [25]:
compass.show()